In [1]:
import pandas as pd

dataset = pd.read_csv('housing.csv')

In [2]:
print(dataset.shape)
dataset[:5]

(506, 13)


,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [3]:
# Move 'medv' column to front
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [4]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [5]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [6]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

2.86.2
s3://sagemaker-us-east-1-608009264409/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-608009264409/boston-housing/input/validation/validation_dataset.csv


In [21]:
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator

region = sess.boto_session.region_name    
container = retrieve('linear-learner', region)
print(container)

ll_estimator = Estimator(container,
    role=get_execution_role() , 
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    use_spot_instances = True, 
    max_run=100,
    max_wait=600
)

ll_estimator.set_hyperparameters(
    predictor_type='regressor', 
    mini_batch_size=32)

382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


In [22]:
from sagemaker import TrainingInput

training_data_channel   = TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = TrainingInput(s3_data=validation_data_path, content_type='text/csv')

ll_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [23]:
ll_estimator.fit(ll_data)

2022-05-17 13:42:51 Starting - Starting the training job...
2022-05-17 13:42:54 Starting - Launching requested ML instancesProfilerReport-1652794971: InProgress
.........
2022-05-17 13:44:39 Starting - Preparing the instances for training.........
2022-05-17 13:46:19 Downloading - Downloading input data...
2022-05-17 13:46:51 Training - Training image download completed. Training in progress.
2022-05-17 13:46:51 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[05/17/2022 13:46:44 INFO 139857910982464] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_me

In [24]:
%%bash -s "$ll_estimator.output_path"
aws s3 ls --recursive $1

2022-05-17 13:46:54        920 boston-housing/output/linear-learner-2022-05-17-13-42-51-383/output/model.tar.gz
2022-05-17 13:46:58          0 boston-housing/output/linear-learner-2022-05-17-13-42-51-383/profiler-output/framework/training_job_end.ts
2022-05-17 13:46:52      93446 boston-housing/output/linear-learner-2022-05-17-13-42-51-383/profiler-output/system/incremental/2022051713/1652795160.algo-1.json
2022-05-17 13:46:58          0 boston-housing/output/linear-learner-2022-05-17-13-42-51-383/profiler-output/system/training_job_end.ts


In [25]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'linear-learner-demo-'+timestamp
print(endpoint_name)

linear-learner-demo-17-13-48-04


In [26]:
ll_predictor = ll_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

-------!

In [27]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [28]:
ll_predictor.serializer = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = ll_predictor.predict(test_sample)
print(response)

[['30.457378387451172']]


In [29]:
test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

response = ll_predictor.predict(test_samples)
print(response)

[['30.457378387451172'], ['24.491931915283203']]


In [30]:
import boto3,json

runtime = boto3.Session().client(service_name='runtime.sagemaker') 

response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv',
                                  Body=test_sample)

print(json.loads(response['Body'].read()))

{'predictions': [{'score': 30.457378387451172}]}


In [ ]:
ll_predictor.delete_endpoint()